In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import urllib.request
import math
import time
from pandas import DataFrame, Series
from urllib.request import Request, urlopen
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import chrome
from selenium.common.exceptions import *
import datetime
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException, StaleElementReferenceException

In [ ]:
# Initialize job data dictionary
jobs = {
    "roles": [],
    "companies": [],
    "locations": [],
    "experience": [],
    "skills": [],
    "Dates": [],
    "PresentDate": [],
    "salary": [],
}


In [ ]:
# Setup for Selenium WebDriver
service = Service('C:/Users/AYUSHI/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe')
driver = webdriver.Chrome(service=service)
driver.implicitly_wait(10)  

In [ ]:
MAX_PAGE_NUM = 2


In [ ]:
base_url = "https://www.naukri.com/jobs-in-india-602?industryTypeIdGid=135&industryTypeIdGid=136&industryTypeIdGid=137&industryTypeIdGid=138&industryTypeIdGid=139&industryTypeIdGid=140&industryTypeIdGid=141&industryTypeIdGid=142&industryTypeIdGid=143&industryTypeIdGid=144&industryTypeIdGid=145&industryTypeIdGid=146&industryTypeIdGid=147"
driver.get(base_url)
time.sleep(3)  

In [ ]:
for page in range(602,902):  # Start from page 301 to 500
    print(f"Scraping page {page}...")
    try:
        # Locate job list
        job_list = driver.find_elements(By.CSS_SELECTOR, ".srp-jobtuple-wrapper .sjw__tuple")

        for job in job_list:
            try:
                # Extract job details
                role = job.find_element(By.CSS_SELECTOR, ".row1 .title").text
                company = job.find_element(By.CSS_SELECTOR, ".row2 .comp-dtls-wrap .comp-name").text
                location = job.find_element(By.CSS_SELECTOR, ".row3 .job-details .loc-wrap .loc span").text
                salaries = job.find_element(By.CSS_SELECTOR, ".row3 .job-details .sal-wrap .sal span").text
                exp = job.find_element(By.CSS_SELECTOR, ".row3 .job-details .exp-wrap .exp span").text
                date = job.find_element(By.CSS_SELECTOR, ".row6 .job-post-day").text
                skills = job.find_element(By.CSS_SELECTOR, ".row5 .tags-gt").text

                # Append job details to dictionary
                today = datetime.datetime.today().strftime('%Y-%m-%d')
                jobs["roles"].append(role)
                jobs["companies"].append(company)
                jobs["locations"].append(location)
                jobs["experience"].append(exp)
                jobs["salary"].append(salaries)
                jobs["Dates"].append(date)
                jobs["PresentDate"].append(today)
                jobs["skills"].append(skills)
            except NoSuchElementException:
                pass  # Skip if any detail is missing

        # Scroll to the "Next" button and click to go to the next page
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//*[@id='lastCompMark']/a[2]/span"))
            )
            # Ensure the button is visible
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            time.sleep(1)  # Allow scroll to complete
            try:
                next_button.click()
            except ElementClickInterceptedException:
                print("Click intercepted. Retrying with JavaScript...")
                driver.execute_script("arguments[0].click();", next_button)  # Force click
            time.sleep(3)  # Allow the next page to load
        except TimeoutException:
            print("No 'Next' button found. Stopping the scraping process.")
            break

    except NoSuchElementException:
        print(f"No more pages available after page {page}. Stopping.")
        break

In [ ]:
driver.quit

In [ ]:
df = pd.DataFrame(jobs)
df.to_csv('Consumer3.csv', index=False)
print(f"Data scraped successfully and saved ")